In [ ]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

parent_dir = os.path.abspath("./..")
sys.path.append(parent_dir)

_=load_dotenv(find_dotenv())

# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
# os.environ['LANGSMITH_TRACING_V2'] = 'true'
# os.environ['LANGSMITH_PROJECT'] = 'deep-research'
# os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'

OPENAI_API_KEY=os.getenv('AZURE_OPENAI_API_KEY_US2') 
os.environ['OPENAI_API_VERSION'] = '2024-12-01-preview'
os.environ['AZURE_OPENAI_ENDPOINT'] = "https://agents-4on.cognitiveservices.azure.com/"
os.environ['AZURE_OPENAI_DEPLOYMENT'] = "gpt-4.1-eus2"

### User Clarification and Brief Generation

*The goal of scoping is to gather user-context needed for research.*

We'll scope the research in two phases:

1. **User Clarification** - Determines if additional clarification is needed from the user
2. **Brief Generation** - Transforms the conversation into a detailed research brief

In [ ]:
from utils import show_prompt
from src.deep_research_from_scratch.prompts import clarify_with_user_instructions
show_prompt(clarify_with_user_instructions, "Clarify with User Instructions")

In [ ]:
from utils import show_prompt
from src.deep_research_from_scratch.prompts import transform_messages_into_research_topic_prompt
show_prompt(transform_messages_into_research_topic_prompt, "Research Topic")

#### State and Schemas

First, we'll define the state object and schemas for our research process. 

The state object serves as our primary mechanism for storing and passing context between different phases of the research workflow. 

We can use it to [write and select context](https://blog.langchain.com/context-engineering-for-agents/) that will be used to guide the research.

In [ ]:
%%writefile ../src/deep_research_from_scratch/state_scope.py

"""State Definitions and Pydantic Schemas for Research Scoping.

This defines the state objects and structured schemas used for
the research agent scoping workflow, including researcher state management and output schemas.
"""

import operator
from typing_extensions import Optional, Annotated, List, Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

# ===== STATE DEFINITIONS =====

class AgentInputState(MessagesState):
    """Input state for the full agent - only contains messages from user input."""
    pass

class AgentState(MessagesState):
    """
    Main state for the full multi-agent research system.
    
    Extends MessagesState with additional fields for research coordination.
    Note: Some fields are duplicated across different state classes for proper
    state management between subgraphs and the main workflow.
    """

    # Research brief generated from user conversation history
    research_brief: Optional[str]
    # Messages exchanged with the supervisor agent for coordination
    supervisor_messages: Annotated[Sequence[BaseMessage], add_messages]
    # Raw unprocessed research notes collected during the research phase
    raw_notes: Annotated[list[str], operator.add] = []
    # Processed and structured notes ready for report generation
    notes: Annotated[list[str], operator.add] = []
    # Final formatted research report
    final_report: str

# ===== STRUCTURED OUTPUT SCHEMAS =====

class ClarifyWithUser(BaseModel):
    """Schema for user clarification decision and questions."""
    
    need_clarification: bool = Field(
        description="Whether the user needs to be asked a clarifying question.",
    )
    question: str = Field(
        description="A question to ask the user to clarify the report scope",
    )
    verification: str = Field(
        description="Verify message that we will start research after the user has provided the necessary information.",
    )

class ResearchQuestion(BaseModel):
    """Schema for structured research brief generation."""
    
    research_brief: str = Field(
        description="A research question that will be used to guide the research.",
    )

#### Scope Research

Now, we'll create a simple workflow to clarify the user's intent and write a research brief.

We'll let the LLM determine whether it has sufficient clarification to write the brief.
 
This will use LangGraph's [Command](https://langchain-ai.github.io/langgraph/how-tos/graph-api/#combine-control-flow-and-state-updates-with-command) to direct the control flow and updating state. The `Command` object takes two key parameters:
- `goto`: Specifies the next node to execute (or `END` to terminate)
- `update`: Dictionary of state updates to apply before transitioning

This pattern allows our functions to both process data and direct the workflow based on their results. 

It creates a more flexible and maintainable system than traditional static graph structures.

In [7]:
%%writefile ../src/deep_research_from_scratch/research_agent_scope.py

"""User Clarification and Research Brief Generation.

This module implements the scoping phase of the research workflow, where we:
1. Assess if the user's request needs clarification
2. Generate a detailed research brief from the conversation

The workflow uses structured output to make deterministic decisions about
whether sufficient context exists to proceed with research.
"""

from datetime import datetime
from typing_extensions import Literal

from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, get_buffer_string
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

from src.deep_research_from_scratch.prompts import clarify_with_user_instructions, transform_messages_into_research_topic_prompt
from src.deep_research_from_scratch.state_scope import AgentState, ClarifyWithUser, ResearchQuestion, AgentInputState

#-Azure-------------------------------------
import os
from langchain_openai import AzureChatOpenAI
#-------------------------------------------

# ===== UTILITY FUNCTIONS =====

def get_today_str() -> str:
    """Get current date in a human-readable format."""
    # return datetime.now().strftime("%a %b %-d, %Y") # Unix
    return datetime.now().strftime("%a %b %#d, %Y") # Windows

# ===== CONFIGURATION =====

# Initialize model

# model = init_chat_model(model="openai:gpt-4.1", temperature=0.0)

model = AzureChatOpenAI(
    api_key = os.getenv('AZURE_OPENAI_API_KEY_US2'),  
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
)

# ===== WORKFLOW NODES =====

def clarify_with_user(state: AgentState) -> Command[Literal["write_research_brief", "__end__"]]:
    """
    Determine if the user's request contains sufficient information to proceed with research.
    
    Uses structured output to make deterministic decisions and avoid hallucination.
    Routes to either research brief generation or ends with a clarification question.
    """
    # Set up structured output model
    structured_output_model = model.with_structured_output(ClarifyWithUser)

    # Invoke the model with clarification instructions
    response = structured_output_model.invoke([
        HumanMessage(content=clarify_with_user_instructions.format(
            messages=get_buffer_string(messages=state["messages"]), 
            date=get_today_str()
        ))
    ])
    
    # Route based on clarification need
    if response.need_clarification:
        return Command(
            goto=END, 
            update={"messages": [AIMessage(content=response.question)]}
        )
    else:
        return Command(
            goto="write_research_brief", 
            update={"messages": [AIMessage(content=response.verification)]}
        )

def write_research_brief(state: AgentState):
    """
    Transform the conversation history into a comprehensive research brief.
    
    Uses structured output to ensure the brief follows the required format
    and contains all necessary details for effective research.
    """
    # Set up structured output model
    structured_output_model = model.with_structured_output(ResearchQuestion)
    
    # Generate research brief from conversation history
    response = structured_output_model.invoke([
        HumanMessage(content=transform_messages_into_research_topic_prompt.format(
            messages=get_buffer_string(state.get("messages", [])),
            date=get_today_str()
        ))
    ])
    
    # Update state with generated research brief and pass it to the supervisor
    return {
        "research_brief": response.research_brief,
        "supervisor_messages": [HumanMessage(content=f"{response.research_brief}.")]
    }

# ===== GRAPH CONSTRUCTION =====

# Build the scoping workflow
deep_researcher_builder = StateGraph(AgentState, input_schema=AgentInputState)

# Add workflow nodes
deep_researcher_builder.add_node("clarify_with_user", clarify_with_user)
deep_researcher_builder.add_node("write_research_brief", write_research_brief)

# Add workflow edges
deep_researcher_builder.add_edge(START, "clarify_with_user")
deep_researcher_builder.add_edge("write_research_brief", END)

# Compile the workflow
scope_research = deep_researcher_builder.compile()

Overwriting ../src/deep_research_from_scratch/research_agent_scope.py


In [8]:
# Compile with in-memory checkpointer to test in notebook
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver
from src.deep_research_from_scratch.research_agent_scope import deep_researcher_builder

checkpointer = InMemorySaver()
scope = deep_researcher_builder.compile(checkpointer=checkpointer)
# display(Image(scope.get_graph(xray=True).draw_mermaid_png()))

In [9]:
# Run the workflow
from utils import format_messages
from langchain_core.messages import HumanMessage

# q = 'I want to research the best coffee shops in San Francisco.'        
# q = 'I am researchin how to develop a multiagent application with sub-agents for RAG, querying an SQL database, and websearch, and a main agent that compliles all the info a final report. I intend to use Python and LangGraph'
q = 'Искам да разбера за ситуацията с Лукойл в България в следствие на санкциите, наложени от администрацията на президента Доналд Тръмп.'   

thread = {"configurable": {"thread_id": "1"}}
result = scope.invoke({"messages": [HumanMessage(content= q )]}, config=thread)
format_messages(result['messages'])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Искам да разбера за ситуацията с Лукойл в България в следствие на санкциите, наложени от администрацията на     │
│ президента Доналд Тръмп.                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Благодаря за информацията. Разбирам, че желаете да получите анализ относно ситуацията с Лукойл в България във   │
│ връзка със санкциите, наложени от администрацията на президента Доналд Тръмп. Започвам проучването по тази      │
│ тема.                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# # Run the workflow
# from utils import format_messages
# from langchain_core.messages import HumanMessage

# q = "Let's examine coffee quality to assess the best coffee shops in San Francisco"

# thread = {"configurable": {"thread_id": "1"}}
# result = scope.invoke({"messages": [HumanMessage(content= q )]}, config=thread)
# format_messages(result['messages'])

In [10]:
from rich.markdown import Markdown
Markdown(result["research_brief"])

I want to understand the situation regarding Lukoil in Bulgaria in the aftermath of sanctions imposed by the       
administration of President Donald Trump. Specifically, I would like a detailed analysis of how these US sanctions 
affected Lukoil's operations, market position, and political or economic relations in Bulgaria. Please include     
details about the timing and scope of the sanctions, any government or corporate responses in Bulgaria, observed   
economic or logistical impacts on Lukoil's activities, and notable shifts in regulatory or public discourse in     
Bulgaria resulting from these sanctions. Unless specified by the available data, consider various periods and all  
types of sanctions imposed by the Trump administration that could have impacted Lukoil in Bulgaria. Prioritize     
reputable sources, including Bulgarian government statements, Lukoil corporate communications, and major           
international news agencies, with a preference for sources in Bulgarian where available.

### Evaluation

Now that we've scoped our research, let's test it with a few examples to make sure that it's working as expected.

Let's think about what makes for a good research brief:

* It captures relevant criteria from the user chat
* It does not invent or assume any criteria that the user did not explicitly provide

Here are two sample input conversations.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

conversation_1 = [
    HumanMessage(content="What's the best way to invest $50,000 for retirement?"),
    AIMessage(content="Could you please provide some additional information to tailor the investment advice for your $50,000 retirement goal? Specifically:\n Your current age or desired retirement age\n Your risk tolerance (low, medium, high)\n Any preferences for investment types (e.g., stocks, bonds, mutual funds, real estate)\n Whether you are investing through a tax-advantaged account (e.g., IRA, 401(k)) or a regular brokerage account\n This will help me provide more personalized and relevant suggestions."),
    HumanMessage(content="I'm 25 and I want to retire by 45. My risk tolerance is high right now but I think will decrease over time. I have heard that stocks and ETFs are a good choice, but I'm open to anything. And I already have a 401k, but this would just be through a regular brokerage account."),
]

conversation_2 = [
    HumanMessage(content="I am looking for an apartment in NYC, can you help me?"),
    AIMessage(content="Could you please specify your apartment preferences? For example:\n Desired neighborhoods or boroughs\n Number of bedrooms/bathrooms\n Budget range (monthly rent)\n Any amenities or must-have features\n Preferred move-in date\n This information will help me provide the most relevant apartment options in NYC."),
    HumanMessage(content="I'd prefer to live in Chelsea, Flatiron, or West Village. I'm looking for a 2 bed 2 bath, and I am looking for monthly rent below 7k. I'd like this to be a doorman building and have an in unit washer and dryer, but it's okay if there's no washer dryer. It's a plus if the building has a gym. And I'd like to move in in September 2025."),
]

Now let's manually write out each criteria from these conversations that we would want preserved by a research brief.

In [ ]:
criteria_1 = [
    "Current age is 25",
    "Desired retirement age is 45",
    "Current risk tolerance is high",
    "Interested in investing in stocks and ETFs",
    "Open to forms of investment beyond stocks and ETFs"
    "Investment account is a regular brokerage account",
]

criteria_2 = [
    "Looking for a 2 bed 2 bath apartment in Chelsea, Flatiron, or West Village",
    "Monthly rent below 7k",
    "Should be in a doorman building",
    "Ideally have an in unit washer and dryer but not strict",
    "Ideally have a gym but not strict",
    "Move in date is September 2025"
]

We're going to use LangSmith to run this experiment. 

Running an experiment in LangSmith comprises of three steps: 

1. Creating the dataset
2. Writing the evaluator(s)
3. Running the experiment

We'll start by creating our dataset and adding our two examples to it.

In [ ]:
import os
from langsmith import Client

# Initialize the LangSmith client
# langsmith_client = Client(api_key=os.getenv("LANGSMITH_API_KEY"))
langsmith_client = Client(api_key=LANGSMITH_API_KEY)

# Create the dataset
dataset_name = "deep_research_scoping"
if not langsmith_client.has_dataset(dataset_name=dataset_name):
    
    # Create the dataset
    dataset = langsmith_client.create_dataset(
        dataset_name=dataset_name,
        description="A dataset that measures the quality of research briefs generated from an input conversation",
    )

    # Add the examples to the dataset
    langsmith_client.create_examples(
        dataset_id=dataset.id,
        examples=[
            {
                "inputs": {"messages": conversation_1},
                "outputs": {"criteria": criteria_1},
            },
            {
                "inputs": {"messages": conversation_2},
                "outputs": {"criteria": criteria_2},
            },
        ],
    )

Now, we need to write an evaluator that will compare our research brief against the success criteria that we have specified for each example. For this, we'll use an LLM-as-judge. You can fine some useful tips for writing llm-as-judge evaluators [here](https://hamel.dev/blog/posts/llm-judge/index.html), which include:

1. **Role Definition with Expertise Context**
   - Defined specific expert roles ("research brief evaluator", "meticulous auditor")  
   - Specialized the role to the specific evaluation domain

2. **Clear Task Specification**
   - Binary pass/fail judgments (avoiding complex multi-dimensional scoring)
   - Explicit task boundaries and objectives
   - Focus on actionable evaluation criteria

3. **Rich Contextual Background**
   - Provide domain-specific context about research brief quality
   - Explain the importance of accurate evaluation
   - Connect evaluation outcomes to downstream consequences

4. **Structured XML Organization**
   - Used semantic XML tags for different sections
   - Clear separation of role, task, context, inputs, guidelines, and outputs
   - Improved prompt parsing and comprehension

5. **Comprehensive Guidelines with Examples**
   - Detailed PASS/FAIL criteria with specific conditions
   - Multiple concrete examples showing correct judgments
   - 3-4 examples per prompt covering different scenarios
   - Both positive and negative examples for each judgment type
   - Edge case handling and decision boundary clarification

6. **Explicit Output Instructions**
   - Clear guidance on how to apply the evaluation criteria
   - Instructions for handling ambiguous cases
   - Emphasis on consistency and systematic evaluation

7. **Bias Reduction Techniques**
   - "Strict but fair" guidance to balance precision and recall
   - "When in doubt, lean toward FAIL" for conservative evaluation
   - Systematic evaluation process to reduce subjective variation

In [ ]:
from src.deep_research_from_scratch.prompts import BRIEF_CRITERIA_PROMPT
show_prompt(BRIEF_CRITERIA_PROMPT, "BRIEF_CRITERIA_PROMPT")

In [ ]:
from typing_extensions import cast
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class Criteria(BaseModel):
    """
    Individual success criteria evaluation result.
    
    This model represents a single evaluation criteria that should be present
    in the research brief, along with a detailed assessment of whether it was
    successfully captured and the reasoning behind that assessment.
    """
    criteria_text: str = Field(
        description="The specific success criteria being evaluated (e.g., 'Current age is 25', 'Monthly rent below 7k')"
    )
    reasoning: str = Field(
        description="Detailed explanation of why this criteria is or isn't captured in the research brief, including specific evidence from the brief"
    )
    is_captured: bool = Field(
        description="Whether this specific criteria is adequately captured in the research brief (True) or missing/inadequately addressed (False)"
    )

def evaluate_success_criteria(outputs: dict, reference_outputs: dict):
    """
    Evaluate whether the research brief captures all required success criteria.
    
    This function evaluates each criterion individually to provide focused assessment
    and detailed reasoning for each evaluation decision.
    
    Args:
        outputs: Dictionary containing the research brief to evaluate
        reference_outputs: Dictionary containing the list of success criteria
        
    Returns:
        Dict with evaluation results including score (0.0 to 1.0)
    """
    research_brief = outputs["research_brief"]
    success_criteria = reference_outputs["criteria"]

    model = ChatOpenAI(model="gpt-4.1", temperature=0)
    structured_output_model = model.with_structured_output(Criteria)
    
    # Run evals
    responses = structured_output_model.batch([
    [
        HumanMessage(
            content=BRIEF_CRITERIA_PROMPT.format(
                research_brief=research_brief,
                criterion=criterion
            )
        )
    ] 
    for criterion in success_criteria])
    
    # Ensure the criteria_text field is populated correctly
    individual_evaluations = [
        Criteria(
            reasoning=response.reasoning,
            criteria_text=criterion,
            is_captured=response.is_captured
        )
        for criterion, response in zip(success_criteria, responses)
    ]
    
    # Calculate overall score as percentage of captured criteria
    captured_count = sum(1 for eval_result in individual_evaluations if eval_result.is_captured)
    total_count = len(individual_evaluations)
    
    return {
        "key": "success_criteria_score", 
        "score": captured_count / total_count if total_count > 0 else 0.0,
        "individual_evaluations": [
            {
                "criteria": eval_result.criteria_text,
                "captured": eval_result.is_captured,
                "reasoning": eval_result.reasoning
            }
            for eval_result in individual_evaluations
        ]
    }

Our second evaluator will check that the research brief does not make any assumptions that the user did not specify in the research brief.

In [ ]:
from src.deep_research_from_scratch.prompts import BRIEF_HALLUCINATION_PROMPT
show_prompt(BRIEF_HALLUCINATION_PROMPT, "BRIEF_HALLUCINATION_PROMPT")

In [ ]:
# Improved NoAssumptions class with reasoning field and enhanced descriptions
class NoAssumptions(BaseModel):
    """
    Evaluation model for checking if research brief makes unwarranted assumptions.
    
    This model evaluates whether the research brief contains any assumptions,
    inferences, or additions that were not explicitly stated by the user in their
    original conversation. It provides detailed reasoning for the evaluation decision.
    """
    no_assumptions: bool = Field(
        description="Whether the research brief avoids making unwarranted assumptions. True if the brief only includes information explicitly provided by the user, False if it makes assumptions beyond what was stated."
    )
    reasoning: str = Field(
        description="Detailed explanation of the evaluation decision, including specific examples of any assumptions found or confirmation that no assumptions were made beyond the user's explicit statements."
    )

def evaluate_no_assumptions(outputs: dict, reference_outputs: dict):
    """
    Evaluate whether the research brief avoids making unwarranted assumptions.
    
    This evaluator checks that the research brief only includes information
    and requirements that were explicitly provided by the user, without
    making assumptions about unstated preferences or requirements.
    
    Args:
        outputs: Dictionary containing the research brief to evaluate
        reference_outputs: Dictionary containing the success criteria for reference
        
    Returns:
        Dict with evaluation results including boolean score and detailed reasoning
    """
    research_brief = outputs["research_brief"]
    success_criteria = reference_outputs["criteria"]
    
    model = ChatOpenAI(model="gpt-4.1", temperature=0)
    structured_output_model = model.with_structured_output(NoAssumptions)
    
    response = structured_output_model.invoke([
        HumanMessage(content=BRIEF_HALLUCINATION_PROMPT.format(
            research_brief=research_brief, 
            success_criteria=str(success_criteria)
        ))
    ])
    
    return {
        "key": "no_assumptions_score", 
        "score": response.no_assumptions,
        "reasoning": response.reasoning
    }

Now that we have our evaluators, we can run our experiment.

In [ ]:
import uuid

def target_func(inputs: dict):
    config = {"configurable": {"thread_id": uuid.uuid4()}}
    return scope.invoke(inputs, config=config)

langsmith_client.evaluate(
    target_func,
    data=dataset_name,
    evaluators=[evaluate_success_criteria, evaluate_no_assumptions],
    experiment_prefix="Deep Research Scoping",
)

Why perform evals like this?

* Ensure that individual steps in your application are doing what you expect.
* With tracing to LangSmith, you also get a sense of how long each step takes as well as the cost.
* You can always try out cheaper, faster models to see if they can get the job done.

If your agent makes mistakes, you can tune the prompts that the agent is provided to try and improve performance as well!